<a href="https://colab.research.google.com/github/harnalashok/hadoop/blob/main/spark_dataframe_expts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Last amended: 30th Sep, 2021
# Myfolder: github/hadoop
# Objectives:
#             i) Install pyspark on colab
#             ii) Install koalas on colab
#
#
# Java 8 install: https://stackoverflow.com/a/58191107
# Hadoop install: https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/SingleCluster.html
# Spark install:  https://stackoverflow.com/a/64183749
#                 https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/

# Full spark install
> At times the download mirror for Apache Spark file `.tgz` is unavailable. If you get an error, please check if the download mirror is OK. <br>
> In case you are also changing the spark or hadoop version, same change will have to be made in the spark environment. 

### 1.0 Libraries

In [1]:
# 1.0 How to set environment variable
import os  
import time  

## 2.0 Define some functions

#### ssh_install()

In [2]:
# 2.0 Function to install ssh client and sshd (Server)
def ssh_install():
  print("\n--1. Download and install ssh server----\n")
  ! sudo apt-get remove openssh-client openssh-server
  ! sudo apt install openssh-client openssh-server
  
  print("\n--2. Restart ssh server----\n")
  ! service ssh restart

#### Java install

In [3]:
# 3.0 Function to download and install java 8
def install_java():
  ! rm -rf /usr/java

  print("\n--Download and install Java 8----\n")
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null        # install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     # set environment variable

  !update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
  !update-alternatives --set javac /usr/lib/jvm/java-8-openjdk-amd64/bin/javac
  
  !mkdir -p /usr/java
  ! ln -s "/usr/lib/jvm/java-8-openjdk-amd64"  "/usr/java"
  ! mv "/usr/java/java-8-openjdk-amd64"  "/usr/java/latest"
  
  !java -version       #check java version
  !javac -version

#### setup ssh passphrase

In [4]:
# 6.0 Function tp setup ssh passphrase
def set_keys():
  print("\n---22. Generate SSH keys----\n")
  ! cd ~ ; pwd 
  ! cd ~ ; ssh-keygen -t rsa -P '' -f ~/.ssh/id_rsa
  ! cd ~ ; cat ~/.ssh/id_rsa.pub >> ~/.ssh/authorized_keys
  ! cd ~ ; chmod 0600 ~/.ssh/authorized_keys


#### Set environment

In [5]:
# 7.0 Function to set up environmental variables
def set_env():
  print("\n---23. Set Environment variables----\n")
  # 'export' command does not work in colab
  # https://stackoverflow.com/a/57240319
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  os.environ["JRE_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/jre"   
  

#### function to install prerequisites
java and ssh<br>


In [6]:
# 8.0 Function to call all functions
def install_components():
  print("\n--Install java----\n")
  ssh_install()
  install_java()  
  #set_keys()
  set_env()


## 3.0 Install components
Start downloading, install and configure. Takes around 2 minutes<br>
Your <u>input *'y'* is required </u>at one place while overwriting earlier ssh keys

In [ ]:
# 9.0 Start installation
start = time.time()
install_components()
end = time.time()
print("\n---Time taken----\n")
print((end- start)/60)

## 4.0 Install spark
koalas will also be installed

### Define functions

`findspark`: PySpark isn't on `sys.path` by default, but that doesn't mean it can't be used as a regular library. You can address this by either symlinking pyspark into your site-packages, or adding `pyspark` to `sys.path` at runtime. `findspark` does the latter.

In [8]:
# 1.0 Function to download and unzip spark
def spark_koalas_install():
  print("\n--1.1 Install findspark----\n")
  !pip install -q findspark

  print("\n--1.2 Install databricks Koalas----\n")
  !pip install koalas

  print("\n--1.3 Download Spark Apache tar.gz----\n")
  #! wget -c https://apachemirror.wuchna.com/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
  ! wget -c https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz 

  print("\n--1.4 Transfer downloaded content and unzip tar.gz----\n")
  !  mv /content/spark*   /opt/
  ! tar -xzf /opt/spark-3.1.2-bin-hadoop3.2.tgz  --directory /opt/

  print("\n--1.5 Check folder for files----\n")
  ! ls -la /opt


In [9]:
# 1.1 Function to set environment
def set_spark_env():
  print("\n---2. Set Environment variables----\n")
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64" 
  os.environ["JRE_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/jre" 
  os.environ["SPARK_HOME"] = "/opt/spark-3.1.2-bin-hadoop3.2"     
  os.environ["LD_LIBRARY_PATH"] += ":/opt/spark-3.1.2-bin-hadoop3.2/lib/native"
  os.environ["PATH"] += ":/opt/spark-3.1.2-bin-hadoop3.2/bin:/opt/spark-3.1.2-bin-hadoop3.2/sbin"
  print("\n---2.1. Check Environment variables----\n")
  # Check
  ! echo $PATH
  ! echo $LD_LIBRARY_PATH

In [10]:
# 1.2 Function to configure spark 
def spark_conf():
  print("\n---3. Configure spark to access hadoop----\n")
  !mv /opt/spark-3.1.2-bin-hadoop3.2/conf/spark-env.sh.template  /opt/spark-3.1.2-bin-hadoop3.2/conf/spark-env.sh
  #!echo "HADOOP_CONF_DIR=/opt/hadoop-3.2.2/etc/hadoop/" >> /opt/spark-3.1.2-bin-hadoop3.2/conf/spark-env.sh
  print("\n---3.1 Check ----\n")
  #!cat /opt/spark-3.1.1-bin-hadoop3.2/conf/spark-env.sh

### Install spark


In [ ]:
spark_koalas_install()

In [12]:
# 2.0 Call all the three functions
def install_spark():
  spark_koalas_install()
  set_spark_env()
  spark_conf()


In [ ]:
# 2.1 
install_spark()

# Create SparkSession
Henceforth use 'spark' as SparkSession object<br>
For Spark configuration options, see [here](http://spark.apache.org/docs/latest/configuration.html#spark-properties)

In [19]:
from pyspark.sql import SparkSession

# Get spark in sys.path
import findspark
findspark.init()

#  Build  spark session
#     with certain configuration options
#     .master => Connect to spark which URL? "local" to run locally, 
#                "local[4]" to run locally with 4 cores,
#                or "spark://master:7077" to run on a Spark standalone cluster.
#
spark = SparkSession. \
                    builder. \
                    master("local[*]"). \
                    config("spark.driver.memory", "3g"). \
                    getOrCreate()


# Test spark
Create a pandas dataframe and then use 'spark' object to transform it to Spark DataFrame


### Call libraries

In [20]:
# 3.0 Just call some libraries to test
import pandas as pd
import numpy as np
import os

# 3.1 Call other spark libraries
#     Just to test
import databricks.koalas as ks
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

### Understanding SparkSession
You can understand more about 'spark' session object using some available methods. <br>
For Spark configuration options, see [here](http://spark.apache.org/docs/latest/configuration.html#spark-properties)

In [21]:
# 3.1.1
# Get spark configuration
spark.conf.get("spark.driver.memory")

'3g'

In [22]:
# 3.1.2 
# Get spark session 
abc = spark.builder.getOrCreate()

### Creating spark dataframe

#### From pandas dataframe

In [23]:
# 4.0 Pandas DataFrame
pdf = pd.DataFrame({
        'x1': ['a','a','b','b', 'b', 'c', 'd','d'],
        'x2': ['apple', 'orange', 'orange','orange', 'peach', 'peach','apple','orange'],
        'x3': [1, 1, 2, 2, 2, 4, 1, 2],
        'x4': [2.4, 2.5, 3.5, 1.4, 2.1,1.5, 3.0, 2.0],
        'y1': [1, 0, 1, 0, 0, 1, 1, 0],
        'y2': ['yes', 'no', 'no', 'yes', 'yes', 'yes', 'no', 'yes']
    })

# 4.1
pdf

,x1,x2,x3,x4,y1,y2
0,a,apple,1,2.4,1,yes
1,a,orange,1,2.5,0,no
2,b,orange,2,3.5,1,no
3,b,orange,2,1.4,0,yes
4,b,peach,2,2.1,0,yes
5,c,peach,4,1.5,1,yes
6,d,apple,1,3.0,1,no
7,d,orange,2,2.0,0,yes


In [24]:
# 4.2 Transform to Spark DataFrame
#     and print
df = spark.createDataFrame(pdf)
df.show()

+---+------+---+---+---+---+
| x1|    x2| x3| x4| y1| y2|
+---+------+---+---+---+---+
|  a| apple|  1|2.4|  1|yes|
|  a|orange|  1|2.5|  0| no|
|  b|orange|  2|3.5|  1| no|
|  b|orange|  2|1.4|  0|yes|
|  b| peach|  2|2.1|  0|yes|
|  c| peach|  4|1.5|  1|yes|
|  d| apple|  1|3.0|  1| no|
|  d|orange|  2|2.0|  0|yes|
+---+------+---+---+---+---+



In [25]:
df1 = abc.createDataFrame(pdf)
df1.show()

+---+------+---+---+---+---+
| x1|    x2| x3| x4| y1| y2|
+---+------+---+---+---+---+
|  a| apple|  1|2.4|  1|yes|
|  a|orange|  1|2.5|  0| no|
|  b|orange|  2|3.5|  1| no|
|  b|orange|  2|1.4|  0|yes|
|  b| peach|  2|2.1|  0|yes|
|  c| peach|  4|1.5|  1|yes|
|  d| apple|  1|3.0|  1| no|
|  d|orange|  2|2.0|  0|yes|
+---+------+---+---+---+---+



In [26]:
############

# Your experiments
Once SparkSession is created as above, begin using the session object 'spark', henceforth, OR get a the same object with a new name using `SparkSession.builder.getOrCreate()` as below.

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [44]:
pathToFolder = "/content/drive/MyDrive/healthcare-analytics/breast_cancer_wisconsin/"
pathToFolder1 = "/content/drive/MyDrive/Colab_data_files/adult/"

In [40]:
# Get existing spark session using builder object
abc = SparkSession.builder.getOrCreate()

In [ ]:
abc.conf.get("spark.driver.memory")

In [42]:
df = spark.read.csv(
                     pathToFolder+"breast_cancer.csv",
                     header = True
                     )

In [ ]:
df.show(5)

In [47]:
len(df.columns)
df.count()

569

In [50]:
df1 = spark.read.csv(
                     pathToFolder1+"adultdata_modified_na.csv",
                     header = True
                     )

In [ ]:
df1.show()

In [52]:
df1 = df.sample(fraction = 0.5)
df2 = df.sample(fraction = 1.0)  # Shuffle it 

In [ ]:
df1.head(4)

In [ ]:
df1.collect()

In [ ]:
len(df1.columns)

In [ ]:
df1.summary().show()

In [ ]:
print(df1.schema)

In [ ]:
df1.describe().show()

In [ ]:
df1.toPandas()

In [ ]:
df1.count()

In [ ]:
df1.crosstab("radius_mean", "texture_mean").show()

In [ ]:
cols = ["radius_mean", "texture_mean"]
df1.select(*cols).show()

In [ ]:
df1.selectExpr("radius_mean * 2" , "texture_mean/3").show()

In [ ]:
df1.selectExpr("radius_mean * texture_mean").alias("abcd").show()

In [ ]:
cols = ["radius_mean", "texture_mean"]
df1.sort(*cols).show()

In [ ]:
df1.take(5)

In [ ]:
df1.tail(5)

In [ ]:
help(df1.take(5))

In [ ]:
fd = df1.to_koalas()

In [ ]:
fd['diagnosis'].value_counts()

In [ ]:
grd = fd.groupby('diagnosis')

In [ ]:
grd['radius_mean'].aggregate(sum)

In [ ]:
grd['radius_mean'].apply(sum)

In [ ]:
df1.agg({'radius_mean': 'sum'}).show()

In [ ]:
grd = df1.groupby('diagnosis')

In [ ]:
grd.agg({'radius_mean': 'sum'}).show()

In [ ]:
df1.limit(10).select('radius_mean').show()

In [ ]:
df1.columns

In [ ]:
df1.withColumnRenamed("symmetry_se", "symmeterySE").limit(10).show()

In [ ]:
c = [ i.replace("_", "") for i in df1.columns]

In [ ]:
c

In [ ]:
df1.toDF(*c).limit(10).show()

In [ ]:
df1.select(*df1.columns[:4]).sort("radius_mean",ascending = False).show()

In [ ]:
from pyspark.sql.functions import mean, sumDistinct

In [ ]:
df1.select(sumDistinct("radius_mean")).show()